In [5]:
import os
from google.cloud import bigquery

In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/viktorbyskov/Desktop/viktorbyskov-analytics/analytics-399003-32aaa8bcb33a.json"

In [8]:
client = bigquery.Client()

In [9]:
sql_query = """SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 3"""

In [10]:
job = client.query(sql_query)

In [11]:
for row in job.result():
    print(row)

Row(('20230918', 1695048275080431, 'sec20_timer', [{'key': 'page_title', 'value': {'string_value': 'Viktor Byskov - Viktor Byskov', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_id', 'value': {'string_value': None, 'int_value': 1695048254, 'float_value': None, 'double_value': None}}, {'key': 'page_location', 'value': {'string_value': 'https://viktorbyskov.dk/', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'session_engaged', 'value': {'string_value': '1', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'engagement_time_msec', 'value': {'string_value': None, 'int_value': 1591, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_number', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}, {'key': 'engaged_session_event', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}], None, None, 777008367, None, None, 

In [12]:
job = client.get_job('bquxjob_72bb18f5_18ab14e942e') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 1000


#START

In [129]:
dataset = "analytics_406290682"
tablename = "events_20231103"

sql = f"""
SELECT * 
FROM `analytics-399003.{dataset}.{tablename}`
LIMIT 1000
"""

In [130]:
results = client.query(sql).to_dataframe()

In [131]:
columns_to_use = ['event_date', 'event_timestamp', 'event_name', 'event_params',
       'event_bundle_sequence_id',
       'user_pseudo_id',
       'user_first_touch_timestamp', 'device', 'geo',
       'traffic_source', 'stream_id', 'platform']

In [132]:
df = results.copy()

In [133]:
df = df[columns_to_use]

In [134]:
e = "event_params_"
df[e + "page_title"] = None
df[e + "ga_session_id"] = None
df[e + "page_location"] = None
df[e + "session_engaged"] = None
df[e + "engagement_time_msec"] = None
df[e + "ga_session_number"] = None
df[e + "engaged_session_event"] = None

for index, row in df.iterrows():
    eventparams = row["event_params"]
    for param in eventparams:
        if param['key'] == 'page_title':
            df.at[index, "event_params_page_title"] = param['value']['string_value']

        elif param['key'] == 'ga_session_id':
            df.at[index, "event_params_ga_session_id"] = param['value']['int_value']

        elif param['key'] == 'page_location':
            df.at[index, "event_params_page_location"] = param['value']['string_value']

        elif param['key'] == 'session_engaged':
            df.at[index, "event_params_session_engaged"] = param['value']['string_value']

        elif param['key'] == 'engagement_time_msec':
            df.at[index, "event_params_engagement_time_msec"] = param['value']['int_value']

        elif param['key'] == 'ga_session_number':
            df.at[index, "event_params_ga_session_number"] = param['value']['int_value']

        elif param['key'] == 'engaged_session_event':
            df.at[index, "event_params_engaged_session_event"] = param['value']['int_value']

            break

In [135]:
df.drop(columns=["event_params", "device", "geo", "traffic_source"], inplace=True)

In [136]:
df.to_csv(f"cleaned_{tablename}.csv", index=False)